In [1]:
# Core Audio Processing
import librosa
import soundfile as sf
import pyAudioAnalysis
from pyAudioAnalysis import audioTrainTest as aT
from pyAudioAnalysis import audioBasicIO
from pyAudioAnalysis import MidTermFeatures

# Numerical Processing
import numpy as np
import pandas as pd

# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Visualization
import matplotlib.pyplot as plt

# System utilities
import os
import warnings
warnings.filterwarnings('ignore')

# will be used to save the model to be used to make future predictions
import joblib

# Emotion (02): Encoded as:
# 01 = neutral
# 02 = calm
# 03 = happy
# 04 = sad
# 05 = angry
# 06 = fearful
# 07 = disgust
# 08 = surprised.

In [2]:
# Define data path
data_path = "data/audio/actor1"
files = []

# Iterate through each file and extract features
for file in os.listdir(data_path):
    if file.endswith(".wav"):
        file_path = os.path.join(data_path, file)
        # Parse labels from filename
        parts = file.split("-")
        emotion = int(parts[2])  # Extract emotion from filename
        actor = int(parts[-1].split(".")[0])  # Extract actor ID
        files.append({"file_path": file_path, "emotion": emotion, "actor": actor})

# Create DataFrame
data = pd.DataFrame(files)
print(data.head(20))

                                     file_path  emotion  actor
0   data/audio/actor1/03-01-06-01-02-02-02.wav        6      2
1   data/audio/actor1/03-01-05-01-02-01-16.wav        5     16
2   data/audio/actor1/03-01-08-01-01-01-14.wav        8     14
3   data/audio/actor1/03-01-06-01-02-02-16.wav        6     16
4   data/audio/actor1/03-01-05-01-02-01-02.wav        5      2
5   data/audio/actor1/03-01-01-01-02-02-06.wav        1      6
6   data/audio/actor1/03-01-02-01-02-01-12.wav        2     12
7   data/audio/actor1/03-01-01-01-02-02-12.wav        1     12
8   data/audio/actor1/03-01-02-01-02-01-06.wav        2      6
9   data/audio/actor1/03-01-02-02-01-01-06.wav        2      6
10  data/audio/actor1/03-01-02-02-01-01-12.wav        2     12
11  data/audio/actor1/03-01-06-02-01-02-16.wav        6     16
12  data/audio/actor1/03-01-05-02-01-01-02.wav        5      2
13  data/audio/actor1/03-01-08-02-02-01-14.wav        8     14
14  data/audio/actor1/03-01-06-02-01-02-02.wav        6

In [3]:
# this will give you the exact type of emotion the audio is displaying

emotion_dict = {
    1: "neutral",
    2: "calm",
    3: "happy",
    4: "sad",
    5: "angry",
    6: "fearful",
    7: "disgust",
    8: "surprised"
}

data["emotion_label"] = data["emotion"].map(emotion_dict)


In [4]:
print(data.head())

                                    file_path  emotion  actor emotion_label
0  data/audio/actor1/03-01-06-01-02-02-02.wav        6      2       fearful
1  data/audio/actor1/03-01-05-01-02-01-16.wav        5     16         angry
2  data/audio/actor1/03-01-08-01-01-01-14.wav        8     14     surprised
3  data/audio/actor1/03-01-06-01-02-02-16.wav        6     16       fearful
4  data/audio/actor1/03-01-05-01-02-01-02.wav        5      2         angry


In [10]:
def extract_features(file_path):
    # Load the audio file
    y, sr = librosa.load(file_path, duration=2.5, offset=0.5)

    # Extract MFCCs
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    mfccs_mean = mfccs.mean(axis=1)
    mfccs_std = mfccs.std(axis=1)

    # Extract Chroma Features
    chroma = librosa.feature.chroma_stft(y=y, sr=sr)
    chroma_mean = chroma.mean(axis=1)

    # Extract Mel Spectrogram
    mel = librosa.feature.melspectrogram(y=y, sr=sr)
    mel_mean = mel.mean(axis=1)

    # Spectral Contrast
    contrast = librosa.feature.spectral_contrast(y=y, sr=sr)
    contrast_mean = contrast.mean(axis=1)

    # Tonnetz
    tonnetz = librosa.feature.tonnetz(y=librosa.effects.harmonic(y), sr=sr)
    tonnetz_mean = tonnetz.mean(axis=1)

    # Zero-Crossing Rate
    zcr = librosa.feature.zero_crossing_rate(y)
    zcr_mean = zcr.mean()

    # Spectral Centroid
    centroid = librosa.feature.spectral_centroid(y=y, sr=sr)
    centroid_mean = centroid.mean()

    # Spectral Bandwidth
    bandwidth = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    bandwidth_mean = bandwidth.mean()

    # Spectral Roll-off
    rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
    rolloff_mean = rolloff.mean()

    # Combine all features into a single array
    features = np.hstack([
        mfccs_mean, mfccs_std, chroma_mean, mel_mean, 
        contrast_mean, tonnetz_mean, zcr_mean, 
        centroid_mean, bandwidth_mean, rolloff_mean
    ])

    return features


In [11]:
print(data.columns)

Index(['file_path', 'emotion', 'actor', 'emotion_label'], dtype='object')


In [12]:

# Prepare features and labels
data["features"] = data["file_path"].apply(lambda x: extract_features(x) if x is not None else np.zeros(30))

X = np.array(data["features"].tolist())
y = data["emotion"]

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [14]:
# Train the model
clf = RandomForestClassifier()
clf.fit(X_train, y_train)

# Evaluate the model
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred, target_names=list(emotion_dict.values())))

joblib.dump(clf, 'emotion_model')

              precision    recall  f1-score   support

     neutral       0.50      0.29      0.37        17
        calm       0.53      0.96      0.68        28
       happy       0.49      0.49      0.49        37
         sad       0.64      0.51      0.57        45
       angry       0.89      0.62      0.73        50
     fearful       0.59      0.48      0.53        33
     disgust       0.38      0.55      0.44        33
   surprised       0.66      0.64      0.65        45

    accuracy                           0.58       288
   macro avg       0.58      0.57      0.56       288
weighted avg       0.61      0.58      0.58       288



['emotion_model']

Understanding the Model Used Chat to get a simple paragraph below

This data evaluates a classification model's performance in predicting emotions, showing metrics like precision, recall, and F1-score for each class (e.g., *happy, sad, angry*). Accuracy (0.61) indicates the model correctly predicted 61% of the 288 samples. Macro averages (precision: 0.60, recall: 0.60, F1: 0.59) show the unweighted mean performance across all classes, treating each equally. Weighted averages (precision: 0.62, recall: 0.61, F1: 0.61) account for class sizes, giving more weight to larger classes. The model performs best on "angry" (F1: 0.77) and struggles with "neutral" and "disgust." Overall, the model has moderate performance, with room for improvement, particularly in underrepresented classes.
